In [49]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
import re
import pandas as pd
from sklearn.svm import LinearSVC

In [50]:
from math import e

from sklearn.preprocessing import MinMaxScaler


class DataExtraction(BaseEstimator, TransformerMixin):
    def __init__(self, features_list):
        self.features_list = features_list
    def fit(self,X,y=None):
        return self
    def transform(self,X):
        retour = X
        for feature_name, feature_function in self.features_list.items():
            retour[feature_name] = X["text"].apply(feature_function)
        retour = retour.drop(columns = "text")
        return retour

def get_dictionnaire(string_list):
    features = {
        "taille_phrase":lambda x: len(x),
        "Nombre_mot":lambda x: len(x.split()),
        "email": lambda x: len(re.findall(r'[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}', x)) > 0,
        "presence_monnaie": lambda x: 1 if re.search(r'[\$\€\£]', x) else 0 ,
        "presence telephone": lambda x: 1 if re.search(r'\b\d{10,}\b', x) else 0,
        "presence_caratere_speciaux": lambda x: 1 if re.search(r'[!@#$%^&*(),.?":{}|<>]', x) else 0,
        "proportion_majuscule": lambda x: sum(1 for c in x if c.isupper()) / len(x) if len(x) > 0 else 0,
        "presence_lien": lambda x: 1 if re.search(r'\b(http|www)\S+', x) else 0
    }
    if string_list == "all":
        return features
    else:
        dict = {}
        try:
            for key in string_list:
                try:
                    dict[key] = features[key]
                except:
                    print(f"La fonction {key} n'existe pas dans le dictionnaire")
        except:
            print(f"La fonction demande une liste de string")
        return dict
            

def GenerateModel(features_names, model, data,vectorizer=CountVectorizer(), scaler=StandardScaler()) :
    target = data["spam"]
    data = data.drop(columns=["spam"])
    featurePipe = Pipeline(steps=[("extraction feature",DataExtraction(features_names)),("inputing",SimpleImputer(strategy="mean")),("scaling",scaler)])
    preparation = ColumnTransformer(transformers=
                              [("features",featurePipe,["text"]),
                               ("vectorisation",vectorizer,"text")]
                              )

    modelPipe = Pipeline(steps=[("prep données",preparation),("model",model)])
    modelPipe.fit(data,target)
    return modelPipe




In [51]:
df = pd.read_csv("BD1.txt",sep="\t",header=None,names=["spam","text"])
y1 = df["spam"]
X1 = df
X1_train, X1_test, y1_train, y1_test = train_test_split(X1,y1 , test_size=0.2, random_state=42)


In [52]:
model = GenerateModel(model=LinearSVC(),data=X1_train, features_names=get_dictionnaire(["taille_phrase", "Nombre_mot", "email", "presence_monnaie", "presence telephone", "presence_caratere_speciaux", "proportion_majuscule", "presence_lien"]))
print(classification_report(y1_test,model.predict(X1_test)))
print(model.predict(pd.DataFrame(['You won 200 billion dollars, call now!', 'Hi, how are you?'], columns=["text"])))
model

              precision    recall  f1-score   support

         ham       0.99      1.00      0.99       966
        spam       0.99      0.93      0.96       149

    accuracy                           0.99      1115
   macro avg       0.99      0.96      0.98      1115
weighted avg       0.99      0.99      0.99      1115

['ham' 'ham']


Pipeline(steps=[('prep données',
                 ColumnTransformer(transformers=[('features',
                                                  Pipeline(steps=[('extraction '
                                                                   'feature',
                                                                   DataExtraction(features_list={'Nombre_mot': <function get_dictionnaire.<locals>.<lambda> at 0x000001E8071D27A0>,
                                                                                                 'email': <function get_dictionnaire.<locals>.<lambda> at 0x000001E8086737E0>,
                                                                                                 'presence telephone': <function get_dictionnaire.<locals>....
                                                                                                 'presence_monnaie': <function get_dictionnaire.<locals>.<lambda> at 0x000001E808673740>,
                                                                                                 'proportion_majuscule': <function get_dictionnaire.<locals>.<lambda> at 0x000001E8086739C0>,
                                                                                                 'taille_phrase': <function get_dictionnaire.<locals>.<lambda> at 0x000001E80492BC40>})),
                                                                  ('inputing',
                                                                   SimpleImputer()),
                                                                  ('scaling',
                                                                   StandardScaler())]),
                                                  ['text']),
                                                 ('vectorisation',
                                                  CountVectorizer(),
                                                  'text')])),
                ('model', LinearSVC())])

In [53]:
from sklearn.ensemble import RandomForestClassifier


model = GenerateModel(model=RandomForestClassifier(),data=X1_train, features_names=get_dictionnaire(["taille_phrase"]))
print(classification_report(y1_test,model.predict(X1_test)))
print(model.predict(pd.DataFrame(['You won 200 billion dollars, call now!', 'Hi, how are you?'], columns=["text"])))
model

              precision    recall  f1-score   support

         ham       0.98      1.00      0.99       966
        spam       1.00      0.86      0.92       149

    accuracy                           0.98      1115
   macro avg       0.99      0.93      0.96      1115
weighted avg       0.98      0.98      0.98      1115

['ham' 'ham']


Pipeline(steps=[('prep données',
                 ColumnTransformer(transformers=[('features',
                                                  Pipeline(steps=[('extraction '
                                                                   'feature',
                                                                   DataExtraction(features_list={'taille_phrase': <function get_dictionnaire.<locals>.<lambda> at 0x000001E804C7DEE0>})),
                                                                  ('inputing',
                                                                   SimpleImputer()),
                                                                  ('scaling',
                                                                   StandardScaler())]),
                                                  ['text']),
                                                 ('vectorisation',
                                                  CountVectorizer(),
                                                  'text')])),
                ('model', RandomForestClassifier())])

## Création de modèle pour les 255 combinaisons de features

### Pour LinearSVC

In [54]:
from itertools import combinations

features = {
    "taille_phrase": lambda x: len(x),
    "Nombre_mot": lambda x: len(x.split()),
    "email": lambda x: len(re.findall(r'[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}', x)) > 0,
    "presence_monnaie": lambda x: 1 if re.search(r'[\$\€\£]', x) else 0,
    "presence telephone": lambda x: 1 if re.search(r'\b\d{10,}\b', x) else 0,
    "presence_caratere_speciaux": lambda x: 1 if re.search(r'[!@#$%^&*(),.?\":{}|<>]', x) else 0,
    "proportion_majuscule": lambda x: sum(1 for c in x if c.isupper()) / len(x) if len(x) > 0 else 0,
    "presence_lien": lambda x: 1 if re.search(r'\b(http|www)\S+', x) else 0
}

# Générer toutes les combinaisons possibles
all_combinations = []
for r in range(1, len(features) + 1):
    combinations_r = list(combinations(features.keys(), r))
    all_combinations.extend(combinations_r)

# Créer un dictionnaire de toutes les combinaisons
combinations_dict = {}
for i, combo in enumerate(all_combinations, 1):
    combinations_dict[f"combination_{i}"] = {k: features[k] for k in combo}

print(f"Nombre total de combinaisons: {len(all_combinations)}")
print(combinations_dict)


Nombre total de combinaisons: 255
{'combination_1': {'taille_phrase': <function <lambda> at 0x000001E808673F60>}, 'combination_2': {'Nombre_mot': <function <lambda> at 0x000001E808673E20>}, 'combination_3': {'email': <function <lambda> at 0x000001E808673D80>}, 'combination_4': {'presence_monnaie': <function <lambda> at 0x000001E808673CE0>}, 'combination_5': {'presence telephone': <function <lambda> at 0x000001E808673EC0>}, 'combination_6': {'presence_caratere_speciaux': <function <lambda> at 0x000001E806EF4040>}, 'combination_7': {'proportion_majuscule': <function <lambda> at 0x000001E806EF40E0>}, 'combination_8': {'presence_lien': <function <lambda> at 0x000001E806EF4180>}, 'combination_9': {'taille_phrase': <function <lambda> at 0x000001E808673F60>, 'Nombre_mot': <function <lambda> at 0x000001E808673E20>}, 'combination_10': {'taille_phrase': <function <lambda> at 0x000001E808673F60>, 'email': <function <lambda> at 0x000001E808673D80>}, 'combination_11': {'taille_phrase': <function <l

In [55]:
import time
from sklearn.metrics import accuracy_score, classification_report

results_df = pd.DataFrame(columns=['combination', 'accuracy', 'time'])

for combination, dict in combinations_dict.items():
    start_time = time.time()
    model = GenerateModel(model=LinearSVC(), data=X1_train, features_names=dict)
    end_time = time.time()
    training_time = end_time - start_time
    accuracy = classification_report(y1_test, model.predict(X1_test), output_dict=True)
    results_df = pd.concat([
        results_df, 
        pd.DataFrame({
            'combination': [list(dict.keys())], 
            'accuracy': [accuracy],
            'time': [training_time]
        })
    ])

results_df


C:\Users\melos\AppData\Local\Temp\ipykernel_15964\1715787905.py:12: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  results_df = pd.concat([
c:\Users\melos\Documents\formation greta\ML\env\Lib\site-packages\sklearn\svm\_base.py:1243: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
c:\Users\melos\Documents\formation greta\ML\env\Lib\site-packages\sklearn\svm\_base.py:1243: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
c:\Users\melos\Documents\formation greta\ML\env\Lib\site-packages\sklearn\svm\_base.py:1243: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
c:\Users\melos\Documents\f

,combination,accuracy,time
0,[taille_phrase],"{'ham': {'precision': 0.9886947584789312, 'rec...",0.186958
0,[Nombre_mot],"{'ham': {'precision': 0.9897540983606558, 'rec...",0.172878
0,[email],"{'ham': {'precision': 0.9887410440122825, 'rec...",0.161284
0,[presence_monnaie],"{'ham': {'precision': 0.9887410440122825, 'rec...",0.127198
0,[presence telephone],"{'ham': {'precision': 0.9887179487179487, 'rec...",0.288871
...,...,...,...
0,"[taille_phrase, Nombre_mot, email, presence te...","{'ham': {'precision': 0.9887179487179487, 'rec...",0.187168
0,"[taille_phrase, Nombre_mot, presence_monnaie, ...","{'ham': {'precision': 0.9887295081967213, 'rec...",0.173942
0,"[taille_phrase, email, presence_monnaie, prese...","{'ham': {'precision': 0.9887295081967213, 'rec...",0.172493
0,"[Nombre_mot, email, presence_monnaie, presence...","{'ham': {'precision': 0.9887295081967213, 'rec...",0.180743


In [56]:
def extract_metrics(row):
    metrics = {}
    for class_name, values in row.items():
        if not isinstance(values, float):
            for metric, value in values.items():
                if class_name in ['ham', 'spam']:
                    metrics[f'{metric}_{class_name}'] = value
    return pd.Series(metrics)

results_df.reset_index(drop=True, inplace=True)
new_columns = results_df["accuracy"].apply(extract_metrics)
df_linearsvc = pd.concat([results_df, new_columns], axis=1)



In [57]:
df

,spam,text
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...
5568,ham,Will ü b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other s..."
5570,ham,The guy did some bitching but I acted like i'd...


### Pour Naive Bayes

In [46]:
from itertools import combinations

features = {
    "taille_phrase": lambda x: len(x),
    "Nombre_mot": lambda x: len(x.split()),
    "email": lambda x: len(re.findall(r'[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}', x)) > 0,
    "presence_monnaie": lambda x: 1 if re.search(r'[\$\€\£]', x) else 0,
    "presence telephone": lambda x: 1 if re.search(r'\b\d{10,}\b', x) else 0,
    "presence_caratere_speciaux": lambda x: 1 if re.search(r'[!@#$%^&*(),.?\":{}|<>]', x) else 0,
    "proportion_majuscule": lambda x: sum(1 for c in x if c.isupper()) / len(x) if len(x) > 0 else 0,
    "presence_lien": lambda x: 1 if re.search(r'\b(http|www)\S+', x) else 0
}
# Générer toutes les combinaisons possibles
all_combinations = []
for r in range(1, len(features) + 1):
    combinations_r = list(combinations(features.keys(), r))
    all_combinations.extend(combinations_r)

# Créer un dictionnaire de toutes les combinaisons
combinations_dict = {}
for i, combo in enumerate(all_combinations, 1):
    combinations_dict[f"combination_{i}"] = {k: features[k] for k in combo}

print(f"Nombre total de combinaisons: {len(all_combinations)}")
print(combinations_dict)


Nombre total de combinaisons: 255
{'combination_1': {'taille_phrase': <function <lambda> at 0x000001E808384900>}, 'combination_2': {'Nombre_mot': <function <lambda> at 0x000001E808384E00>}, 'combination_3': {'email': <function <lambda> at 0x000001E80445D6C0>}, 'combination_4': {'presence_monnaie': <function <lambda> at 0x000001E8071D0A40>}, 'combination_5': {'presence telephone': <function <lambda> at 0x000001E8071D1A80>}, 'combination_6': {'presence_caratere_speciaux': <function <lambda> at 0x000001E8071D07C0>}, 'combination_7': {'proportion_majuscule': <function <lambda> at 0x000001E8071D0C20>}, 'combination_8': {'presence_lien': <function <lambda> at 0x000001E8071D02C0>}, 'combination_9': {'taille_phrase': <function <lambda> at 0x000001E808384900>, 'Nombre_mot': <function <lambda> at 0x000001E808384E00>}, 'combination_10': {'taille_phrase': <function <lambda> at 0x000001E808384900>, 'email': <function <lambda> at 0x000001E80445D6C0>}, 'combination_11': {'taille_phrase': <function <l

In [47]:
from sklearn.metrics import accuracy_score, classification_report
from sklearn.naive_bayes import MultinomialNB

results_df = pd.DataFrame(columns=['combination', 'accuracy'])
for combination, dict in combinations_dict.items():
    model = GenerateModel(model=MultinomialNB(),data=X1_train, features_names=dict, scaler=MinMaxScaler())
    accuracy = classification_report(y1_test,model.predict(X1_test), output_dict=True)
    results_df = pd.concat([results_df, pd.DataFrame({'combination': [list(dict.keys())], 'accuracy': [accuracy]})])
    
results_df

,combination,accuracy
0,[taille_phrase],"{'ham': {'precision': 0.9907692307692307, 'rec..."
0,[Nombre_mot],"{'ham': {'precision': 0.9907692307692307, 'rec..."
0,[email],"{'ham': {'precision': 0.9907692307692307, 'rec..."
0,[presence_monnaie],"{'ham': {'precision': 0.9917864476386037, 'rec..."
0,[presence telephone],"{'ham': {'precision': 0.9917864476386037, 'rec..."
...,...,...
0,"[taille_phrase, Nombre_mot, email, presence te...","{'ham': {'precision': 0.9917864476386037, 'rec..."
0,"[taille_phrase, Nombre_mot, presence_monnaie, ...","{'ham': {'precision': 0.9917864476386037, 'rec..."
0,"[taille_phrase, email, presence_monnaie, prese...","{'ham': {'precision': 0.9917864476386037, 'rec..."
0,"[Nombre_mot, email, presence_monnaie, presence...","{'ham': {'precision': 0.9917864476386037, 'rec..."


In [48]:
def extract_metrics(row):
    metrics = {}
    for class_name, values in row.items():
        if not isinstance(values, float):
            for metric, value in values.items():
                if class_name in ['ham', 'spam']: 
                    metrics[f'{metric}_{class_name}'] = value
    return pd.Series(metrics)

results_df.reset_index(drop=True, inplace=True)
new_columns = results_df["accuracy"].apply(extract_metrics)
df_naive_bayes = pd.concat([results_df, new_columns], axis=1)